# Import Necessary Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import math
import collections
import os
import json
import pickle

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import warnings
warnings.filterwarnings('ignore')

# Load the Train Test Split

In [2]:
X_train = np.load(os.path.join("..", "IPD", "x_train.npy"))
X_test = np.load(os.path.join("..", "IPD", "x_test.npy"))
Y_train = np.load(os.path.join("..", "IPD", "y_train.npy"))
Y_test = np.load(os.path.join("..", "IPD", "y_test.npy"))
print("Training samples : {} \nTesting Samples : {}".format(X_train.shape, X_test.shape))
print("Training Labels : {} \nTesting Labels : {}".format(Y_train.shape, Y_test.shape))

Training samples : (332, 250, 12) 
Testing Samples : (83, 250, 12)
Training Labels : (332,) 
Testing Labels : (83,)


In [3]:
_ , num_timeframes, num_features = X_train.shape

# Reshape data for the model
-----------------
Samples are a 2D matrix which need to be reshaped to 1D to feed into the model

In [4]:
X_train = X_train.reshape((-1,num_timeframes*num_features))
X_test = X_test.reshape((-1,num_timeframes*num_features))
print("Training samples : {} \nTesting Samples : {}".format(X_train.shape, X_test.shape))
print("Training Labels : {} \nTesting Labels : {}".format(Y_train.shape, Y_test.shape))

Training samples : (332, 3000) 
Testing Samples : (83, 3000)
Training Labels : (332,) 
Testing Labels : (83,)


# Grid search over Random Forest model

In [13]:
parameters = {'bootstrap': [False], 
              'oob_score': [False],
              'max_depth': [12, 14, 16],
              'max_features': ['sqrt','log2',None],
              'n_estimators': [16,18,20]}

grid = GridSearchCV(RandomForestClassifier(), parameters, cv=3, verbose=2)
grid.fit(X_train, Y_train)

Fitting 3 folds for each of 27 candidates, totalling 81 fits
[CV] bootstrap=False, max_depth=12, max_features=sqrt, n_estimators=16, oob_score=False 
[CV]  bootstrap=False, max_depth=12, max_features=sqrt, n_estimators=16, oob_score=False, total=   0.1s
[CV] bootstrap=False, max_depth=12, max_features=sqrt, n_estimators=16, oob_score=False 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


[CV]  bootstrap=False, max_depth=12, max_features=sqrt, n_estimators=16, oob_score=False, total=   0.2s
[CV] bootstrap=False, max_depth=12, max_features=sqrt, n_estimators=16, oob_score=False 
[CV]  bootstrap=False, max_depth=12, max_features=sqrt, n_estimators=16, oob_score=False, total=   0.1s
[CV] bootstrap=False, max_depth=12, max_features=sqrt, n_estimators=18, oob_score=False 
[CV]  bootstrap=False, max_depth=12, max_features=sqrt, n_estimators=18, oob_score=False, total=   0.1s
[CV] bootstrap=False, max_depth=12, max_features=sqrt, n_estimators=18, oob_score=False 
[CV]  bootstrap=False, max_depth=12, max_features=sqrt, n_estimators=18, oob_score=False, total=   0.2s
[CV] bootstrap=False, max_depth=12, max_features=sqrt, n_estimators=18, oob_score=False 
[CV]  bootstrap=False, max_depth=12, max_features=sqrt, n_estimators=18, oob_score=False, total=   0.2s
[CV] bootstrap=False, max_depth=12, max_features=sqrt, n_estimators=20, oob_score=False 
[CV]  bootstrap=False, max_depth=12

[CV]  bootstrap=False, max_depth=14, max_features=None, n_estimators=16, oob_score=False, total=   5.0s
[CV] bootstrap=False, max_depth=14, max_features=None, n_estimators=16, oob_score=False 
[CV]  bootstrap=False, max_depth=14, max_features=None, n_estimators=16, oob_score=False, total=   4.7s
[CV] bootstrap=False, max_depth=14, max_features=None, n_estimators=16, oob_score=False 
[CV]  bootstrap=False, max_depth=14, max_features=None, n_estimators=16, oob_score=False, total=   4.5s
[CV] bootstrap=False, max_depth=14, max_features=None, n_estimators=18, oob_score=False 
[CV]  bootstrap=False, max_depth=14, max_features=None, n_estimators=18, oob_score=False, total=   5.6s
[CV] bootstrap=False, max_depth=14, max_features=None, n_estimators=18, oob_score=False 
[CV]  bootstrap=False, max_depth=14, max_features=None, n_estimators=18, oob_score=False, total=   5.4s
[CV] bootstrap=False, max_depth=14, max_features=None, n_estimators=18, oob_score=False 
[CV]  bootstrap=False, max_depth=14

[Parallel(n_jobs=1)]: Done  81 out of  81 | elapsed:  2.4min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [14]:
params = grid.best_params_
best_estimator = grid.best_estimator_
print(params)
print(best_estimator)

{'bootstrap': False, 'max_depth': 16, 'max_features': 'log2', 'n_estimators': 20, 'oob_score': False}
RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=16, max_features='log2',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=20,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


In [15]:
clf = best_estimator
clf.fit(X_train,Y_train)

RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=16, max_features='log2',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=20,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [16]:
Y_pred = clf.predict(X_test)
print(classification_report(Y_test, Y_pred))
print(confusion_matrix(Y_test,Y_pred))

              precision    recall  f1-score   support

           0       0.87      1.00      0.93        13
           1       0.92      0.80      0.86        15
           2       0.81      0.93      0.87        14
           3       1.00      1.00      1.00         9
           4       0.88      0.75      0.81        20
           5       0.77      0.83      0.80        12

    accuracy                           0.87        83
   macro avg       0.88      0.89      0.88        83
weighted avg       0.87      0.87      0.87        83

[[13  0  0  0  0  0]
 [ 2 12  1  0  0  0]
 [ 0  1 13  0  0  0]
 [ 0  0  0  9  0  0]
 [ 0  0  2  0 15  3]
 [ 0  0  0  0  2 10]]


# Train on entire Data

In [17]:
X_train = np.load(os.path.join("..", "IPD", "x_train.npy"))
X_test = np.load(os.path.join("..", "IPD", "x_test.npy"))
Y_train = np.load(os.path.join("..", "IPD", "y_train.npy"))
Y_test = np.load(os.path.join("..", "IPD", "y_test.npy"))

X_train = np.concatenate((X_train,X_test), axis=0)
Y_train = np.concatenate((Y_train,Y_test), axis=0)

_ , num_timeframes, num_features = X_train.shape
X_train = X_train.reshape((-1,num_timeframes*num_features))
from scipy.sparse import coo_matrix
X_sparse = coo_matrix(X_train)

from sklearn.utils import shuffle
X_train, X_sparse, Y_train = shuffle(X_train, X_sparse, Y_train)
X_train = X_train.reshape((-1,num_timeframes,num_features))

In [18]:
X_train = X_train.reshape((-1,num_timeframes*num_features))
X_test = X_test.reshape((-1,num_timeframes*num_features))
print("Training samples : {} \nTesting Samples : {}".format(X_train.shape, X_test.shape))
print("Training Labels : {} \nTesting Labels : {}".format(Y_train.shape, Y_test.shape))

Training samples : (415, 3000) 
Testing Samples : (83, 3000)
Training Labels : (415,) 
Testing Labels : (83,)


In [19]:
parameters = {'bootstrap': [False], 
              'oob_score': [False],
              'max_depth': [14, 16, 18],
              'max_features': ['sqrt','log2',None],
              'n_estimators': [15, 20, 25]}

grid = GridSearchCV(RandomForestClassifier(), parameters, cv=3, verbose=2)
grid.fit(X_train, Y_train)

Fitting 3 folds for each of 27 candidates, totalling 81 fits
[CV] bootstrap=False, max_depth=14, max_features=sqrt, n_estimators=15, oob_score=False 
[CV]  bootstrap=False, max_depth=14, max_features=sqrt, n_estimators=15, oob_score=False, total=   0.2s
[CV] bootstrap=False, max_depth=14, max_features=sqrt, n_estimators=15, oob_score=False 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


[CV]  bootstrap=False, max_depth=14, max_features=sqrt, n_estimators=15, oob_score=False, total=   0.2s
[CV] bootstrap=False, max_depth=14, max_features=sqrt, n_estimators=15, oob_score=False 
[CV]  bootstrap=False, max_depth=14, max_features=sqrt, n_estimators=15, oob_score=False, total=   0.2s
[CV] bootstrap=False, max_depth=14, max_features=sqrt, n_estimators=20, oob_score=False 
[CV]  bootstrap=False, max_depth=14, max_features=sqrt, n_estimators=20, oob_score=False, total=   0.2s
[CV] bootstrap=False, max_depth=14, max_features=sqrt, n_estimators=20, oob_score=False 
[CV]  bootstrap=False, max_depth=14, max_features=sqrt, n_estimators=20, oob_score=False, total=   0.2s
[CV] bootstrap=False, max_depth=14, max_features=sqrt, n_estimators=20, oob_score=False 
[CV]  bootstrap=False, max_depth=14, max_features=sqrt, n_estimators=20, oob_score=False, total=   0.2s
[CV] bootstrap=False, max_depth=14, max_features=sqrt, n_estimators=25, oob_score=False 
[CV]  bootstrap=False, max_depth=14

[CV]  bootstrap=False, max_depth=16, max_features=log2, n_estimators=25, oob_score=False, total=   0.1s
[CV] bootstrap=False, max_depth=16, max_features=None, n_estimators=15, oob_score=False 
[CV]  bootstrap=False, max_depth=16, max_features=None, n_estimators=15, oob_score=False, total=   5.6s
[CV] bootstrap=False, max_depth=16, max_features=None, n_estimators=15, oob_score=False 
[CV]  bootstrap=False, max_depth=16, max_features=None, n_estimators=15, oob_score=False, total=   6.4s
[CV] bootstrap=False, max_depth=16, max_features=None, n_estimators=15, oob_score=False 
[CV]  bootstrap=False, max_depth=16, max_features=None, n_estimators=15, oob_score=False, total=   6.2s
[CV] bootstrap=False, max_depth=16, max_features=None, n_estimators=20, oob_score=False 
[CV]  bootstrap=False, max_depth=16, max_features=None, n_estimators=20, oob_score=False, total=   3.7s
[CV] bootstrap=False, max_depth=16, max_features=None, n_estimators=20, oob_score=False 
[CV]  bootstrap=False, max_depth=16

[Parallel(n_jobs=1)]: Done  81 out of  81 | elapsed:  2.7min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [20]:
clf = grid.best_estimator_
clf.fit(X_train,Y_train)

RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=16, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=25,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

# Saving model

In [21]:
pickle.dump(clf, open(os.path.join("..", "IPD", "RandomForest_model.pkl"), 'wb'))
print("Model Saved")

Model Saved
